# LlamaIndex - Starter Tutorial (Local Models)

This tutorial is based on LlamaIndexofficial docs tutorial ([https://docs.llamaindex.ai/en/stable/getting_started/starter_example_local/](https://docs.llamaindex.ai/en/stable/getting_started/starter_example_local/#starter-tutorial-local-models)):

## Download data and folder setup

On the Docker host side, run the following to set up the `jetson-containers`' `/data` directory.

Your directory structure should look like this:

```
└── ./data/documents/L4T-README
    ├── README-usb-dev-mode.txt
    ├── README-vnc.txt
    └── README-wifi.txt
```

In [ ]:
!ls -lR /data/documents/

> If not, run the following cell to copy the files.

In [ ]:
!mkdir -p /data/documents/L4T-README
!cp ./L4T-README/*.txt /data/documents/L4T-README

> Or run the following commands **on Docker host side (natively on Jetson)**

```
cd jetson-containers
mkdir -p data/documents/L4T-README
cp /media/jetson/L4T-README/*.txt data/documents/L4T-README/
```

## Download the Llama2 model

Check if you have Llama2 13B model downloaded.

In [ ]:
!ollama list

> If not, run the following cell to download the Llama2 model using `ollama` command.

In [ ]:
!ollama pull llama2:13b

## Load data and build an index

Enable DEBUG output.

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.embeddings import resolve_embed_model
from llama_index.llms.ollama import Ollama

import logging
import sys
# import llama_index.core

# llama_index.core.set_global_handler("simple")
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
reader = SimpleDirectoryReader(input_dir="/data/documents/L4T-README/")

In [ ]:
documents = reader.load_data()

In [ ]:
len(documents)

In [ ]:
# bge embedding model
Settings.embed_model = resolve_embed_model("local:BAAI/bge-large-en-v1.5")

In [ ]:
# ollama
Settings.llm = Ollama(model="llama2:13b", request_timeout=300.0)

In [ ]:
# Enlarge the chunk size from the default 1024
Settings.chunk_size = 4096
Settings.chunk_overlap = 200

In [ ]:
index = VectorStoreIndex.from_documents(documents)

You can check what's indexed by saving in `storage` directory.

In [ ]:
index.storage_context.persist()

## Query your data

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("What IPv4 address Jetson device gets assigned when connected to a PC with a USB cable? \
    And what file to edit in order to change the IP address to be assigned to Jetson itself in USB device mode? \
    Plesae state which section you find the answer for each question.")

Let's check the response.

In [ ]:
from IPython.display import Markdown, display

display(Markdown(f"<b>{response}</b>"))